## COMP 3400: Data Preparation Techniques Project
## Analysis of popular movies from 1980 to 2019

**Group Members:**
  - Liudmila Strelnikova 201819885
  - David Chicas 201919354
  
 ## Part One: Changes From Iteration One

In this project submission you can find a PDF that addresses all the instrutor's comments about the changes we can make to Iteration 1 to improve it. Those changes have already been addressed in this iteration.

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import datetime
import string
import pandas as pd
import jinja2

import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import FunctionTransformer

pd.options.mode.chained_assignment = None #chained assignments are not flagged

In [ ]:
movies2 = pd.read_csv("movies2.csv")

In [ ]:
movies2.info()

In [ ]:
movies2.isnull().sum().sort_values(ascending=False)

## Part Two: Data Scaling Pre-Assessment


In [ ]:
ndf = movies2[['score', 'runtime', 'gross']].dropna()
ndf.info()

In [ ]:
#The function plots graphs in a grid whose dimentions are 2 x (number of attributes).
#A row in the grid consits of two sets of axes. The first set of axes has a histogram of an numeric
#attribute, along with its density function. The second set of axes consists of a histogram
#and density plot of a normalized version of the attribute. The method of normalization is set outsite
#the function.

# @df is the dataframe with numeric attributes 
# @columns is an array with the names of columns in the df
# @df_transformed is a dataframe with normalized attributes of df
# @fit_title is the overall title of all the plots

def density_plots(df, columns, df_transformed, fig_title, constrained_layout=True):
    fig, axs = plt.subplots(len(columns),2,figsize=(16,18))
    fig.suptitle(fig_title, fontsize=16)
    axs = axs.flatten()
    i = 0 
    for c in columns:
        ndf[c].hist(ax=axs[i], density=True) # normalizes the density
        ndf[c].plot.density(ax=axs[i], title=c)
        df_transformed[c].hist(ax=axs[i+1], density=True, stacked=True) # normalizes the density
        df_transformed[c].plot.density(ax=axs[i+1], title=str(c+'_transformed'))
        i+=2

val_cols_names = ['score','runtime','gross']

scaler = MinMaxScaler()
min_max_tdf = scaler.fit_transform(ndf)
min_max_tdf = pd.DataFrame(min_max_tdf, index=ndf.index, columns=ndf.columns)

density_plots(ndf, val_cols_names, min_max_tdf, 'MinMax Scaler')

In [ ]:
scaler = MaxAbsScaler()
maxabs_tdf = scaler.fit_transform(ndf)
maxabs_tdf = pd.DataFrame(maxabs_tdf, index=ndf.index, columns=ndf.columns)

density_plots(ndf, val_cols_names, maxabs_tdf, 'MaxAbs Scaler')

In [ ]:
scaler = RobustScaler()
robust_tdf = scaler.fit_transform(ndf)
robust_tdf = pd.DataFrame(robust_tdf, index=ndf.index, columns=ndf.columns)

density_plots(ndf, val_cols_names, robust_tdf, 'Robust Scaler')

In [ ]:
from sklearn.preprocessing import QuantileTransformer

# scaler = QuantileTransformer(n_quantiles=10, random_state=0)
scaler = QuantileTransformer(n_quantiles=10, random_state=0, output_distribution='normal')
qtl_tdf = scaler.fit_transform(ndf)
qtl_tdf = pd.DataFrame(qtl_tdf, index=ndf.index, columns=ndf.columns)

density_plots(ndf, val_cols_names, qtl_tdf, 'Quantile Scaler')

## Standardization

In [ ]:
from sklearn.preprocessing import Normalizer

scaler = Normalizer()
zscore_tdf = scaler.fit_transform(ndf)
zscore_tdf = pd.DataFrame(zscore_tdf, index=ndf.index, columns=ndf.columns)

density_plots(ndf, val_cols_names, zscore_tdf, 'Z-score Scaler')

In [ ]:
from sklearn.preprocessing import FunctionTransformer
import numpy as np

scaler = FunctionTransformer(np.log1p)
log_tdf = scaler.fit_transform(ndf)
log_tdf = pd.DataFrame(log_tdf, index=ndf.index, columns=ndf.columns)

density_plots(ndf, val_cols_names, log_tdf, 'Log Scaler')

We can plot all the normalization and standardization plots togethe

In [ ]:
def plot_histograms_density_for_scalers(df, columns, fig_title, scalers, scaler_names):
    fig, axs = plt.subplots(len(columns),len(scalers)+1,figsize=(16,18),constrained_layout=True)
    fig.suptitle(fig_title, fontsize=16)
    axs = axs.flatten()
    i = 0 
    for c in columns:
        ndf[c].hist(ax=axs[i], density=True) # normalizes the density
        ndf[c].plot.density(ax=axs[i], title=c)
        i+=1
        for j in range(len(scalers)):
            df_transformed = scalers[j].fit_transform(df)
            df_transformed = pd.DataFrame(df_transformed, index=df.index, columns=df.columns)
            df_transformed[c].hist(ax=axs[i], density=True, stacked=True) # normalizes the density
            df_transformed[c].plot.density(ax=axs[i], title=scaler_names[j])
            i+=1
            

val_cols_names = ['score','runtime','gross']
scaler_names = ['MinMax', 'Z-score', 'MaxAbs', 'Robust', 'Quantile', 'Log']
arr_scalers = [MinMaxScaler(), Normalizer(), MaxAbsScaler(), RobustScaler(), QuantileTransformer(n_quantiles=10, random_state=0, output_distribution='normal'),FunctionTransformer(np.log1p)]
plot_histograms_density_for_scalers(ndf, val_cols_names, 'Scalers effect on data', arr_scalers, scaler_names)

From the charts above, it is clear that the only normalization technicque that worked on the gross attribute is quantile transform scaler (QTS). QTS maps the variable's probability distribution to another probablility distribution, and performs well with unimodal data, which includes the distribution of gross values.

Score has a normal distribution with minimal skewness, so min-max scaler works well for this attribute.

Runtime is a bit more skewed than score, so robust scaler is the best choice for this attribute's normalization. Robust scaler removes the median, and scales data according to 